In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import WeightedRandomSampler, Subset

from pytorch_msssim import ssim  # pip install pytorch-msssim

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_curve,
    auc,
    average_precision_score,
    precision_recall_fscore_support,
    precision_recall_curve,
)
from PIL import Image
import random
import matplotlib.pyplot as plt
import numpy as np

import json

from collections import Counter

from timm.models import create_model

### Modelo

In [2]:
class SwinTransformerAutoencoder(nn.Module):
    """
    SwinTransformerAutoencoder
    Autoencoder basado en un codificador Swin Transformer preentrenado y un decodificador convolucional.
    Permite la reconstrucción de imágenes y la detección de anomalías mediante el error de reconstrucción.
    Args:
        pretrained (bool): Si True, utiliza pesos preentrenados para el Swin Transformer.
    Attributes:
        encoder (nn.Module): Codificador basado en Swin Transformer.
        decoder (nn.Module): Decodificador convolucional para reconstrucción de imágenes.
        scoring_criterion (callable or None): Función de pérdida utilizada para el scoring consistente.
    Methods:
        set_scoring_criterion(criterion):
            Establece la función de pérdida para el scoring consistente durante la detección de anomalías.
        forward(x):
            Realiza la pasada hacia adelante del autoencoder.
            Args:
                x (Tensor): Imagen de entrada de tamaño [batch_size, C, H, W].
            Returns:
                Tuple[Tensor, Tensor]: Imagen reconstruida y representación latente.
        detectar_anomalia(x):
            Calcula el error de reconstrucción (MSE) entre la imagen original y la reconstruida para detección de anomalías.
            Args:
                x (Tensor): Imagen de entrada.
            Returns:
                Tensor: Error de reconstrucción por muestra.
        detectar_anomalia_consistente(x):
            Calcula el error de reconstrucción usando la función de pérdida definida (scoring_criterion).
            Si no se ha definido, utiliza MSE por defecto.
            Args:
                x (Tensor): Imagen de entrada.
            Returns:
                Tensor: Error de reconstrucción consistente por muestra.
    """
    def __init__(self, pretrained=True):
        super(SwinTransformerAutoencoder, self).__init__()

        # Codificador: Swin transformer preentrenado
        self.encoder = create_model(
            'swin_tiny_patch4_window7_224',
            pretrained=pretrained,
            num_classes=0
        )

        # Decodificador
        self.decoder = DecodificadorConvolucional()
        
        # AGREGAR: Función de scoring para detección consistente
        self.scoring_criterion = None

    # Método para establecer el criterio de scoring
    def set_scoring_criterion(self, criterion):
        """
        Establece el criterio de evaluación (scoring) para el modelo.

        Parámetros:
            criterion (callable): Función que recibe las predicciones y los valores reales, y devuelve una métrica de evaluación.

        Ejemplo:
            >>> model.set_scoring_criterion(mean_squared_error)
        """

        self.scoring_criterion = criterion

    def forward(self, x):
        # Obtener representación latente
        latente = self.encoder(x)  # [batch_size, 768]

        # Reconstruir la imagen
        reconstruida = self.decoder(latente)
        return reconstruida, latente
    
    # Metodo para detectar anomalías
    # Usar MSE como error de reconstrucción
    def detectar_anomalia(self, x):
        """
        Detecta anomalías calculando el error de reconstrucción entre la entrada y su reconstrucción.

        Parámetros:
            x (torch.Tensor): Tensor de entrada con dimensiones (batch_size, canales, alto, ancho).

        Devuelve:
            torch.Tensor: Error de reconstrucción (MSE) para cada elemento del batch.
        """

        reconstruida, _ = self.forward(x)

        # Calcular error de reconstrucción como MSE
        error_reconstruccion = torch.mean((x - reconstruida) ** 2, dim=[1, 2, 3])
        return error_reconstruccion
    
    # Método para scoring consistente
    def detectar_anomalia_consistente(self, x):
        """
        Detecta anomalías utilizando el mismo criterio de puntuación (función de pérdida) empleado durante el entrenamiento.
        Parámetros:
            x (torch.Tensor): Entrada a evaluar para detectar anomalías.
        Devuelve:
            torch.Tensor: Puntuación de error combinada que indica el grado de anomalía de la entrada.
        Notas:
            - Si no se ha definido un criterio de puntuación (`scoring_criterion`), utiliza el método alternativo `detectar_anomalia` basado en MSE.
            - La puntuación devuelta es consistente con la función de pérdida usada en el entrenamiento del modelo.
        """

        if self.scoring_criterion is None:
            # Fallback al MSE si no hay criterio definido
            return self.detectar_anomalia(x)
            
        reconstruida, _ = self.forward(x)
        
        # Usar la función combinada para scoring (consistente con entrenamiento)
        combined_error, _ = self.scoring_criterion(reconstruida, x)
        return combined_error


class DecodificadorConvolucional(nn.Module):
    """
    DecodificadorConvolucional es una red neuronal basada en PyTorch diseñada para decodificar un vector latente en una imagen RGB de tamaño 224x224. Utiliza una arquitectura de decodificación progresiva con bloques convolucionales y upsampling, seguida de un refinamiento final para mejorar la calidad de la imagen generada.
    Parámetros:
        dim_latente (int): Dimensión del vector latente de entrada. Por defecto es 768.
    Atributos:
        proyeccion_inicial (nn.Sequential): Proyección inicial del vector latente a un mapa de características 8x8.
        decodificador_bloque1-4 (BloqueDecodificador): Bloques convolucionales para upsampling progresivo y reducción de canales.
        upsample_final (nn.Upsample): Upsampling final para ajustar la salida a 224x224 píxeles.
        refinamiento (nn.Sequential): Bloque de capas convolucionales y de normalización para refinar la imagen.
        conv_final (nn.Sequential): Capa final para mapear a 3 canales RGB y aplicar activación sigmoide.
    Métodos:
        forward(x):
            Realiza el paso hacia adelante de la red, decodificando el vector latente x en una imagen RGB de tamaño [batch_size, 3, 224, 224].
    """

    def __init__(self, dim_latente=768):
        super(DecodificadorConvolucional, self).__init__()

        # Proyección inicial mejorada con más capacidad
        self.proyeccion_inicial = nn.Sequential(
            nn.Linear(dim_latente, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512 * 8 * 8),  # Empezar en 8x8 en lugar de 7x7
            nn.ReLU(inplace=True),
            nn.Dropout(0.2)
        )

        # Bloques decodificadores mejorados con más canales
        # 8x8 -> 16x16
        self.decodificador_bloque1 = BloqueDecodificador(512, 384, upsample=True)
        
        # 16x16 -> 32x32  
        self.decodificador_bloque2 = BloqueDecodificador(384, 256, upsample=True)
        
        # 32x32 -> 64x64
        self.decodificador_bloque3 = BloqueDecodificador(256, 128, upsample=True)
        
        # 64x64 -> 128x128
        self.decodificador_bloque4 = BloqueDecodificador(128, 64, upsample=True)
        
        # 128x128 -> 224x224 (upsampling con interpolación)
        self.upsample_final = nn.Upsample(size=(224, 224), mode='bilinear', align_corners=False)
        
        # Refinamiento final con múltiples capas
        self.refinamiento = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 8, kernel_size=3, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True)
        )
        
        # Capa final para generar RGB
        self.conv_final = nn.Sequential(
            nn.Conv2d(8, 3, kernel_size=1),  # 1x1 conv para mapear a RGB
            nn.Sigmoid()
        )

    def forward(self, x):
        # Proyección inicial
        x = self.proyeccion_inicial(x)  # [batch_size, 512*8*8]
        x = x.view(x.size(0), 512, 8, 8)  # [batch_size, 512, 8, 8]

        # Decodificación progresiva
        x = self.decodificador_bloque1(x)  # [batch_size, 384, 16, 16]
        x = self.decodificador_bloque2(x)  # [batch_size, 256, 32, 32] 
        x = self.decodificador_bloque3(x)  # [batch_size, 128, 64, 64]
        x = self.decodificador_bloque4(x)  # [batch_size, 64, 128, 128]
        
        # Upsampling final a 224x224
        x = self.upsample_final(x)  # [batch_size, 64, 224, 224]
        
        # Refinamiento
        x = self.refinamiento(x)  # [batch_size, 8, 224, 224]
        
        # Generar imagen final
        x = self.conv_final(x)  # [batch_size, 3, 224, 224]

        return x

class BloqueDecodificador(nn.Module):
    """
    BloqueDecodificador implementa un bloque de decodificación para redes neuronales, típicamente usado en arquitecturas de segmentación o autoencoders.
    Parámetros:
        in_channels (int): Número de canales de entrada.
        out_channels (int): Número de canales de salida.
        upsample (bool, opcional): Si es True, aplica upsampling usando ConvTranspose2d. Si es False, usa una Conv2d 1x1. Por defecto es True.
    Atributos:
        upsample_layer (nn.Module): Capa utilizada para upsampling o ajuste de canales.
        upsample_bn (nn.BatchNorm2d): Normalización por lotes después del upsampling.
        conv_block (nn.Sequential): Bloque residual con dos capas convolucionales, batch normalization y activación ReLU.
        relu (nn.ReLU): Función de activación ReLU.
        attention (nn.Sequential): Mecanismo simple de atención de canales usando pooling global y convoluciones 1x1.
        dropout (nn.Dropout2d): Capa de dropout para regularización.
    Proceso en forward:
        1. Aplica upsampling o ajuste de canales a la entrada.
        2. Pasa el resultado por un bloque convolucional residual.
        3. Aplica un mecanismo de atención de canales.
        4. Suma la identidad upsampleada (conexión residual).
        5. Aplica activación ReLU y dropout.
        6. Devuelve el tensor procesado.
    Retorna:
        torch.Tensor: Tensor de salida tras decodificación, atención y regularización.
    """
        
    def __init__(self, in_channels, out_channels, upsample=True):
        super(BloqueDecodificador, self).__init__()

        self.upsample = upsample

        if upsample:
            # Usar ConvTranspose2d más efectiva
            self.upsample_layer = nn.ConvTranspose2d(
                in_channels, out_channels, 
                kernel_size=4, stride=2, padding=1, bias=False
            )
            self.upsample_bn = nn.BatchNorm2d(out_channels)
        else:
            self.upsample_layer = nn.Conv2d(in_channels, out_channels, 1)
            self.upsample_bn = nn.BatchNorm2d(out_channels)

        # Bloque residual mejorado
        self.conv_block = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        
        # Conexión residual
        self.relu = nn.ReLU(inplace=True)
        
        # Attention mechanism simple
        self.attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels, out_channels // 4, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels // 4, out_channels, 1),
            nn.Sigmoid()
        )
        
        # Dropout para regularización
        self.dropout = nn.Dropout2d(0.1)

    def forward(self, x):
        # Upsampling
        identity = self.upsample_layer(x)
        identity = self.upsample_bn(identity)
        identity = self.relu(identity)
        
        # Bloque residual
        out = self.conv_block(identity)
        
        # Attention
        att = self.attention(out)
        out = out * att
        
        # Conexión residual
        out += identity
        out = self.relu(out)
        out = self.dropout(out)

        return out

In [3]:
# Función de perdida combinada 
class CombinedReconstructionLoss(nn.Module):
    """
    CombinedReconstructionLoss combina las pérdidas MSE, SSIM y L1 para tareas de reconstrucción de imágenes.
    Parámetros:
        mse_weight (float, opcional): Peso para la pérdida de error cuadrático medio (MSE). Por defecto 1.0.
        ssim_weight (float, opcional): Peso para la pérdida de índice de similitud estructural (SSIM). Por defecto 0.15.
        l1_weight (float, opcional): Peso para la pérdida L1 (error absoluto medio). Por defecto 0.05.
    Forward (entrada):
        reconstructed (Tensor): Imagen reconstruida.
        original (Tensor): Imagen original (objetivo).
    Devuelve:
        total_loss (Tensor): Suma ponderada de las pérdidas MSE, SSIM y L1.
        loss_dict (dict): Diccionario con los valores individuales de cada pérdida:
            - 'mse': Pérdida MSE (float)
            - 'ssim': Pérdida SSIM (float)
            - 'l1': Pérdida L1 (float)
            - 'total': Pérdida total combinada (float)
    Notas:
        - SSIM se calcula como (1 - valor SSIM) para usarlo como término de pérdida.
        - Se asume que todas las pérdidas se calculan sobre tensores con valores en [0, 1].
    """
    
    def __init__(self, mse_weight=1.0, ssim_weight=0.15, l1_weight=0.05):
        super(CombinedReconstructionLoss, self).__init__()
        self.mse_weight = mse_weight
        self.ssim_weight = ssim_weight  
        self.l1_weight = l1_weight
        
    def forward(self, reconstructed, original):
        # MSE Loss (pérdida principal)
        mse_loss = F.mse_loss(reconstructed, original)
        
        # SSIM Loss (preserva estructura)
        ssim_val = ssim(reconstructed, original, data_range=1.0)
        ssim_loss = 1 - ssim_val
        
        # L1 Loss (ayuda con detalles finos)
        l1_loss = F.l1_loss(reconstructed, original)
        
        # Combinar pérdidas
        total_loss = (self.mse_weight * mse_loss + 
                     self.ssim_weight * ssim_loss + 
                     self.l1_weight * l1_loss)
        
        return total_loss, {
            'mse': mse_loss.item(),
            'ssim': ssim_loss.item(), 
            'l1': l1_loss.item(),
            'total': total_loss.item()
        }

### Funciones y clases auxiliares


In [4]:
# Dataset y DataLoader para MVTec AD
class MVTecDataset(Dataset):
    """
    MVTecDataset es un Dataset de PyTorch para cargar el dataset MVTec Anomaly Detection.
    Parámetros:
        root_path (str): Directorio raíz del dataset MVTec.
        category (str): Nombre de la categoría de objeto (por ejemplo, 'bottle', 'capsule').
        is_train (bool, opcional): Si es True, carga el set de entrenamiento (solo imágenes normales). Si es False, carga el set de test (imágenes normales y anómalas). Por defecto es True.
        transform (callable, opcional): Transformaciones a aplicar a las imágenes.
        mask_transform (callable, opcional): Transformaciones a aplicar a las máscaras de ground truth.
    Atributos:
        image_paths (list): Lista de rutas de archivos de imagen.
        labels (list o np.ndarray): Lista o array de etiquetas (0 para normal, 1 para anomalía).
        mask_paths (list o None): Lista de rutas de máscaras o None si no aplica.
    Métodos:
        __len__(): Devuelve la cantidad de muestras en el dataset.
        __getitem__(idx): Devuelve una tupla (imagen, etiqueta, máscara) para el índice idx.
            - imagen (Tensor): Imagen transformada.
            - etiqueta (float): 0 para normal, 1 para anomalía.
            - máscara (Tensor): Tensor de máscara (todo ceros si no está disponible).
    """
     
    def __init__(self, root_path, category, is_train=True, transform=None, mask_transform=None):
        self.root_path = root_path
        self.category = category
        self.is_train = is_train
        self.transform = transform
        self.mask_transform = mask_transform
        
        # Definir directorios
        if self.is_train:
            self.image_dir = os.path.join(root_path, category, 'train', 'good')
            self.image_paths = [os.path.join(self.image_dir, f) for f in os.listdir(self.image_dir) 
                               if f.endswith('.png')]
            self.labels = np.zeros(len(self.image_paths), dtype=np.float32)  # 0 = normal
            self.mask_paths = None
            
        else:  # Test set
            self.image_dir = os.path.join(root_path, category, 'test')
            self.image_paths = []
            self.labels = []
            self.mask_paths = []
            
            # Imágenes normales (buenas)
            good_dir = os.path.join(self.image_dir, 'good')
            if os.path.exists(good_dir):
                good_images = [os.path.join(good_dir, f) for f in os.listdir(good_dir) 
                              if f.endswith('.png')]
                self.image_paths.extend(good_images)
                self.labels.extend([0] * len(good_images))  # 0 = normal
                self.mask_paths.extend([None] * len(good_images))
            
            # Imágenes anómalas (con defectos)
            defect_types = [d for d in os.listdir(self.image_dir) 
                           if os.path.isdir(os.path.join(self.image_dir, d)) and d != 'good']
            
            for defect in defect_types:
                defect_dir = os.path.join(self.image_dir, defect)
                defect_images = [os.path.join(defect_dir, f) for f in os.listdir(defect_dir) 
                                if f.endswith('.png')]
                self.image_paths.extend(defect_images)
                self.labels.extend([1] * len(defect_images))  # 1 = anomalía
                
                # Añadir máscaras de ground truth (si existen)
                gt_dir = os.path.join(root_path, category, 'ground_truth', defect)
                if os.path.exists(gt_dir):
                    for img_path in defect_images:
                        img_name = os.path.basename(img_path)
                        mask_name = img_name.replace('.png', '_mask.png')
                        mask_path = os.path.join(gt_dir, mask_name)
                        if os.path.exists(mask_path):
                            self.mask_paths.append(mask_path)
                        else:
                            self.mask_paths.append(None)
                else:
                    self.mask_paths.extend([None] * len(defect_images))
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Cargar imagen
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB') # Convertir a RGB
        label = self.labels[idx]
    
        # Cargar máscara si existe (solo para test y anomalías)
        mask = None
        if not self.is_train and self.mask_paths[idx] is not None: # Si es test y hay máscara
            mask_path = self.mask_paths[idx]
            mask = Image.open(mask_path).convert('L') # Convertir a escala de grises
            if self.mask_transform:
                mask = self.mask_transform(mask) # Aplicar transformaciones a la máscara
            elif self.transform:
                mask = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                ])(mask) # Aplicar transformaciones por defecto a la máscara
        else:
            # Crear una máscara vacía si no existe
            mask = torch.zeros((1, 224, 224))
    
        # Aplicar transformaciones a la imagen
        if self.transform:
            image = self.transform(image)
    
        # Siempre devolver tres elementos
        return image, label, mask

In [5]:
def split_test_datasets(test_datasets, categories, test_size=0.5, random_state=42):
    """
    Divide cada dataset en `test_datasets` en subconjuntos de validación y test, manteniendo la proporción de muestras normales y anómalas cuando es posible.
    Args:
        test_datasets (list): Lista de datasets a dividir. Cada dataset debe tener un atributo `labels` con las etiquetas de clase (0 para normal, 1 para anomalía).
        categories (list): Lista de nombres de categoría correspondiente a cada dataset en `test_datasets`.
        test_size (float, opcional): Proporción del dataset a incluir en el split de test. Por defecto es 0.5.
        random_state (int, opcional): Semilla aleatoria para reproducibilidad. Por defecto es 42.
    Returns:
        tuple: Una tupla con dos listas:
            - val_datasets (list): Lista de subconjuntos de validación para cada dataset de entrada.
            - final_test_datasets (list): Lista de subconjuntos de test para cada dataset de entrada.
    Notas:
        - Si no es posible hacer un split estratificado (por ejemplo, solo hay una clase), se realiza un split aleatorio simple.
        - Imprime un resumen de la división para cada categoría, incluyendo el conteo total, normales, anómalas, validación y test.
    """
    
    val_datasets = []
    final_test_datasets = []
    
    print("\n" + "="*60)
    print("DIVISIÓN DE DATASETS DE TEST")
    print("="*60)
    
    for i, test_dataset in enumerate(test_datasets):
        # Obtener índices del dataset
        indices = list(range(len(test_dataset)))
        
        # Dividir índices manteniendo la proporción de anomalías
        labels = [test_dataset.labels[j] for j in indices]
        
        # Contar normales y anomalías
        normal_count = sum(1 for l in labels if l == 0)
        anomaly_count = sum(1 for l in labels if l == 1)
        
        try:
            val_indices, test_indices = train_test_split(
                indices, 
                test_size=test_size,
                stratify=labels,  # Mantener proporción de normales/anómalas
                random_state=random_state
            )
        except ValueError:
            # Si no se puede estratificar (ej: solo una clase), hacer split simple
            val_indices, test_indices = train_test_split(
                indices, 
                test_size=test_size,
                random_state=random_state
            )
        
        # Crear subsets
        val_subset = Subset(test_dataset, val_indices)
        test_subset = Subset(test_dataset, test_indices)
        
        val_datasets.append(val_subset)
        final_test_datasets.append(test_subset)
        
        print(f"{categories[i]:12} | Total: {len(indices):3} | "
              f"Normal: {normal_count:3} | Anomalía: {anomaly_count:3} | "
              f"Val: {len(val_subset):3} | Test: {len(test_subset):3}")
    
    print("="*60)
    return val_datasets, final_test_datasets

In [6]:
def crear_sampler_balanceado(dataset, method='weighted'):
    """
    Crea un sampler o un conjunto de índices balanceados para un dataset con clases desbalanceadas.
    Parámetros
    ----------
    dataset : torch.utils.data.Dataset o torch.utils.data.ConcatDataset
        El dataset del que se extraerán las etiquetas para balancear las clases.
        Se asume que cada elemento del dataset retorna una tupla donde el segundo elemento es la etiqueta (0: normal, 1: anomalía).
    method : str, opcional (por defecto 'weighted')
        Método de balanceo a utilizar:
            - 'weighted': utiliza WeightedRandomSampler para muestreo ponderado.
            - 'oversample': duplica la clase minoritaria hasta igualar la mayoritaria.
            - 'undersample': reduce la clase mayoritaria hasta igualar la minoritaria.
    Retorna
    -------
    sampler : torch.utils.data.WeightedRandomSampler o None
        Sampler balanceado para DataLoader si se usa el método 'weighted'. En otros métodos retorna None.
    balanced_indices : list o None
        Lista de índices balanceados para DataLoader si se usa 'oversample' o 'undersample'. En el método 'weighted' retorna None.
    Notas
    -----
    - Imprime la distribución original de clases y detalles del método de balanceo utilizado.
    - Para 'oversample' y 'undersample', los índices pueden ser usados con torch.utils.data.SubsetRandomSampler.
    """
    # Extraer todas las etiquetas del dataset
    if hasattr(dataset, 'datasets'):  # ConcatDataset
        labels = []
        for i in range(len(dataset)):
            _, label, _ = dataset[i]
            labels.append(int(label))
    else:  # Dataset simple
        labels = []
        for i in range(len(dataset)):
            _, label, _ = dataset[i]
            labels.append(int(label))
    
    labels = np.array(labels)
    
    # Contar clases
    class_counts = Counter(labels)
    print(f"   Distribución original: Normal={class_counts[0]}, Anomalía={class_counts[1]}")
    
    if method == 'weighted':
        # Weighted Random Sampler
        class_weights = {}
        total_samples = len(labels)
        
        for class_id, count in class_counts.items():
            class_weights[class_id] = total_samples / (len(class_counts) * count)
        
        sample_weights = [class_weights[int(label)] for label in labels]
        sampler = WeightedRandomSampler(
            weights=sample_weights, 
            num_samples=len(sample_weights),
            replacement=True
        )
        
        print(f"   Método: Weighted Sampling")
        print(f"   Pesos: Normal={class_weights[0]:.3f}, Anomalía={class_weights[1]:.3f}")
        
        return sampler, None
    
    elif method == 'oversample':
        # Oversampling: duplicar la clase minoritaria
        max_count = max(class_counts.values())
        balanced_indices = []
        
        for class_id in [0, 1]:  # normal, anomaly
            class_indices = np.where(labels == class_id)[0]
            # Repetir indices hasta alcanzar max_count
            repeats = max_count // len(class_indices)
            remainder = max_count % len(class_indices)
            
            balanced_indices.extend(class_indices.tolist() * repeats)
            balanced_indices.extend(class_indices[:remainder].tolist())
        
        np.random.shuffle(balanced_indices)
        
        print(f"   Método: Oversampling")
        print(f"   Tamaño balanceado: {len(balanced_indices)} (Normal={max_count}, Anomalía={max_count})")
        
        return None, balanced_indices
    
    elif method == 'undersample':
        # Undersampling: reducir la clase mayoritaria
        min_count = min(class_counts.values())
        balanced_indices = []
        
        for class_id in [0, 1]:
            class_indices = np.where(labels == class_id)[0]
            # Tomar solo min_count muestras
            selected_indices = np.random.choice(class_indices, min_count, replace=False)
            balanced_indices.extend(selected_indices.tolist())
        
        np.random.shuffle(balanced_indices)
        
        print(f"   Método: Undersampling")
        print(f"   Tamaño balanceado: {len(balanced_indices)} (Normal={min_count}, Anomalía={min_count})")
        
        return None, balanced_indices


In [7]:
def crear_dataloader_balanceado(dataset, batch_size, method='weighted', num_workers=4):    
    """
    Crea un DataLoader balanceado para entrenamiento con datasets desbalanceados.
    Esta función crea un DataLoader que maneja el desbalance de clases usando
    muestreo ponderado o muestreo por subconjunto basado en el método especificado.
    Args:
        dataset: Objeto dataset de PyTorch que contiene los datos de entrenamiento
        batch_size (int): Número de muestras por lote
        method (str, opcional): Método de balanceo a usar. Opciones:
            - 'weighted': Usa WeightedRandomSampler para balancear clases
            - Otros valores: Usa Subset con índices balanceados
            Por defecto es 'weighted'
        num_workers (int, opcional): Número de procesos worker para carga de datos.
            Por defecto es 4
    Returns:
        DataLoader: Un DataLoader de PyTorch configurado con el método de balanceo
        y parámetros especificados
    Nota:
        Esta función depende de `crear_sampler_balanceado()` para generar el
        sampler apropiado o índices para balancear el dataset.
    """
    sampler, indices = crear_sampler_balanceado(dataset, method=method)
    
    if sampler is not None:
        # Usar WeightedRandomSampler
        return DataLoader(
            dataset, 
            batch_size=batch_size, 
            sampler=sampler,
            num_workers=num_workers
        )
    else:
        # Usar Subset con indices balanceados
        from torch.utils.data import Subset
        balanced_dataset = Subset(dataset, indices)
        return DataLoader(
            balanced_dataset, 
            batch_size=batch_size, 
            shuffle=True,
            num_workers=num_workers
        )

### Funciones de entrenamiento

In [8]:
def train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, output_dir, logs_dir, model_name):
    """
    Entrena un modelo autoencoder para detección de anomalías usando función de pérdida combinada.
    Esta función realiza el entrenamiento y validación de un modelo autoencoder, utilizando una
    función de pérdida combinada (MSE + SSIM + L1) tanto para entrenamiento como para puntuación
    consistente de anomalías. El modelo se evalúa usando la métrica ROC AUC y se guarda el modelo
    con mejor rendimiento.
    Args:
        model: El modelo autoencoder a entrenar. Debe tener el método set_scoring_criterion.
        train_loader: DataLoader para datos de entrenamiento. Retorna (imágenes, etiquetas, máscaras).
        val_loader: DataLoader para datos de validación. Retorna (imágenes, etiquetas, máscaras).
        criterion: Función de pérdida combinada que retorna (pérdida, diccionario_componentes_pérdida).
        optimizer: Optimizador de PyTorch para los parámetros del modelo.
        scheduler: Planificador de tasa de aprendizaje que toma la pérdida de validación como entrada.
        num_epochs (int): Número de épocas de entrenamiento.
        device: Dispositivo de PyTorch (cpu/cuda) para el modelo y los datos.
        output_dir (str): Ruta del directorio para guardar el checkpoint del mejor modelo.
        logs_dir (str): Ruta del directorio para logs de TensorBoard.
        model_name (str): Identificador de nombre para el modelo (usado en logging y guardado).
    Returns:
        model: El modelo entrenado con los mejores pesos cargados.
    La función realiza las siguientes operaciones:
    - Configura el logging de TensorBoard para monitoreo integral del entrenamiento
    - Entrena el modelo usando pérdida combinada (MSE + SSIM + L1)
    - Valida el modelo y calcula puntuaciones de anomalía usando la misma función de pérdida
    - Rastrea la métrica ROC AUC para el rendimiento de detección de anomalías
    - Guarda el mejor modelo basado en el AUC de validación más alto
    - Registra métricas detalladas, componentes de pérdida y reconstrucciones de muestra
    - Actualiza la tasa de aprendizaje usando el planificador
    El bucle de entrenamiento incluye logging detallado de:
    - Componentes individuales de pérdida (MSE, SSIM, L1, Total)
    - Puntuaciones ROC AUC para evaluación de detección de anomalías
    - Progresión de la tasa de aprendizaje
    - Distribuciones de puntuación para muestras normales vs anómalas
    - Reconstrucciones de imágenes de muestra para inspección visual
    """

    writer = SummaryWriter(log_dir=f'{logs_dir}/{model_name}') # Para tensorboard
    os.makedirs(output_dir, exist_ok=True)

    # Configurar el criterio de scoring en el modelo
    model.set_scoring_criterion(criterion)

    best_auc = 0.0
    best_val_loss = float('inf')
    patience = 201 #25
    patience_counter = 0
    early_stop = False

    print(f"\n INICIANDO ENTRENAMIENTO")
    print(f"   Épocas: {num_epochs}")
    print(f"   Función de pérdida: MSE + SSIM + L1")
    print(f"   Dispositivo: {device}")
    print("="*60)

    for epoch in range(num_epochs):
        # =============
        # Entrenamiento
        # =============
        model.train()
        train_losses = {'total': 0.0, 'mse': 0.0, 'ssim': 0.0, 'l1': 0.0}
        num_train_batches = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} (train)"):
            images, _, _ = batch  # Desempacar correctamente (imagen, etiqueta, máscara)
            images = images.to(device)

            # Forward pass
            reconstructed, _ = model(images)

            # Calcular pérdida combinada
            loss, loss_components = criterion(reconstructed, images)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Acumular pérdidas para logging
            for key in train_losses:
                train_losses[key] += loss_components[key]
            num_train_batches += 1
        
        # ==========
        # Validación
        # ==========
        model.eval()
        val_losses = {'total': 0.0, 'mse': 0.0, 'ssim': 0.0, 'l1': 0.0}
        all_scores = []
        all_labels = []
        num_val_batches = 0

        with torch.no_grad():
            for images, labels, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} (val)"):
                images = images.to(device)
                reconstructed, _ = model(images)

                # Calcular pérdida de validación usando función combinada
                val_loss, val_loss_components = criterion(reconstructed, images)

                # Acumular pérdidas
                for key in val_losses:
                    val_losses[key] += val_loss_components[key]
                num_val_batches += 1

                # Scoring Cconsistente: Usar función combinada para anomaly detection
                batch_scores = []
                for i in range(images.size(0)):
                    single_img = images[i:i+1]  # Mantener dimensión batch
                    single_rec = reconstructed[i:i+1]

                    # Usar la misma función de pérdida para scoring
                    combined_error, _ = criterion(single_rec, single_img)
                    batch_scores.append(combined_error.item())

                all_scores.extend(batch_scores)
                all_labels.extend(labels.numpy())

        # =====================================
        # Calcular métricas y registrar logging
        # =====================================

        # Calcular promedios de pérdidas
        avg_train_losses = {k: v / num_train_batches for k, v in train_losses.items()}
        avg_val_losses = {k: v / num_val_batches for k, v in val_losses.items()}

        # Calcular ROC AUC si tenemos ambas clases (normal y anomalía)
        fpr, tpr, _ = roc_curve(all_labels, all_scores)
        roc_auc = auc(fpr, tpr)

        # Guardar el mejor modelo basado en AUC
        if roc_auc > best_auc:
            best_auc = roc_auc
            torch.save(model.state_dict(), os.path.join(output_dir, f'{model_name}.pth'))
            patience_counter = 0  # Resetear contador de paciencia
            print(f"      Nuevo mejor modelo guardado! AUC: {roc_auc:.4f}")
        else:
            patience_counter += 1
            print(f"      AUC no mejorado. Paciencia: {patience_counter}/{patience}")

        # Logging detallado
        print(f"\nEpoch {epoch+1}/{num_epochs}:")
        print(f"   Train Loss:")
        print(f"     Total: {avg_train_losses['total']:.4f} | MSE: {avg_train_losses['mse']:.4f} | SSIM: {avg_train_losses['ssim']:.4f} | L1: {avg_train_losses['l1']:.4f}")
        print(f"   Val Loss:")
        print(f"     Total: {avg_val_losses['total']:.4f} | MSE: {avg_val_losses['mse']:.4f} | SSIM: {avg_val_losses['ssim']:.4f} | L1: {avg_val_losses['l1']:.4f}")
        print(f"   ROC AUC (Combined Scoring): {roc_auc:.4f} | Best: {best_auc:.4f}")

        # Guardo los valores en tensorboard
        # Logging de pérdidas de entrenamiento
        writer.add_scalar('Loss/Train_Total', avg_train_losses['total'], epoch)
        writer.add_scalar('Loss/Train_MSE', avg_train_losses['mse'], epoch)
        writer.add_scalar('Loss/Train_SSIM', avg_train_losses['ssim'], epoch)
        writer.add_scalar('Loss/Train_L1', avg_train_losses['l1'], epoch)

        # Logging de pérdidas de validación
        writer.add_scalar('Loss/Val_Total', avg_val_losses['total'], epoch)
        writer.add_scalar('Loss/Val_MSE', avg_val_losses['mse'], epoch)
        writer.add_scalar('Loss/Val_SSIM', avg_val_losses['ssim'], epoch)
        writer.add_scalar('Loss/Val_L1', avg_val_losses['l1'], epoch)

        # Logging de métricas de evaluación
        writer.add_scalar('Metrics/ROC_AUC', roc_auc, epoch)
        writer.add_scalar('Metrics/Best_AUC', best_auc, epoch)

        # Logging del learning rate
        writer.add_scalar('Training/Learning_Rate', optimizer.param_groups[0]['lr'], epoch)

        # Logging de histogramas de scores para análisis más detallado
        writer.add_histogram("Scores/Anomaly_Scores", torch.tensor(all_scores), epoch)

        # Separar scores por clase para mejor análisis
        normal_scores = [
            score for score, label in zip(all_scores, all_labels) if label == 0
        ]
        anomaly_scores = [
            score for score, label in zip(all_scores, all_labels) if label == 1
        ]

        if normal_scores:
            writer.add_histogram("Scores/Normal_Scores", torch.tensor(normal_scores), epoch)
        if anomaly_scores:
            writer.add_histogram(
                "Scores/Anomaly_Scores", torch.tensor(anomaly_scores), epoch
            )

        # Guardo imágenes de ejemplo cada cierto número de épocas
        if epoch % 10 == 0 or epoch == num_epochs - 1:
            # Tomar las primeras 8 imágenes del batch de validación para visualización
            with torch.no_grad():
                sample_images = images[:8]
                sample_reconstructed = reconstructed[:8]
        
                # Crear una grilla con imágenes originales y reconstruidas
                comparison = torch.cat([sample_images, sample_reconstructed], dim=0)
                writer.add_images('Images/Original_vs_Reconstructed', comparison, epoch, dataformats='NCHW')

        writer.flush()

        # Actualizar learning rate
        #scheduler.step(avg_val_losses['total']) # Usar el scheduler con la pérdida de validación plateau
        scheduler.step() # Para usar el scheduler sin pasarle la pérdida

        print(f"   Learning Rate: {optimizer.param_groups[0]['lr']:.2e}")
        print("-" * 60)

        # Verificar early stopping
        if patience_counter >= patience:
            print(f"\n Early stopping en época {epoch+1}")
            break

    print(f"\n ENTRENAMIENTO COMPLETADO!")
    print(f"   Mejor AUC alcanzado: {best_auc:.4f}")
    print("="*60)

    return model

### Funciones de Visualización

In [9]:
def visualize_examples(images, labels, scores, masks, threshold, output_dir, model_type):
    """
    Genera visualizaciones de las predicciones del modelo categorizadas por resultados de clasificación.
    Esta función crea ejemplos visuales de Verdaderos Positivos, Verdaderos Negativos, Falsos Positivos
    y Falsos Negativos, junto con una matriz de confusión. Guarda todas las visualizaciones en el
    directorio de salida especificado.
    Args:
        images (array-like): Imágenes de entrada para visualizar. Puede ser array numpy o lista.
                           Forma esperada: (N, 3, H, W) o (N, H, W, 3)
        labels (array-like): Etiquetas verdaderas (0 para normal, 1 para anomalía)
        scores (array-like): Puntuaciones/probabilidades de predicción del modelo
        masks (array-like): Máscaras verdaderas para localización de anomalías (opcional)
        threshold (float): Umbral de decisión para convertir puntuaciones a predicciones binarias
        output_dir (str): Ruta del directorio donde se guardarán las visualizaciones
        model_type (str): Tipo de modelo que se está evaluando (para propósitos de documentación)
    Returns:
        None: La función guarda visualizaciones en disco e imprime información de progreso
    Efectos Secundarios:
        - Crea subdirectorio 'examples' en output_dir
        - Guarda archivos PNG para cada categoría de predicción (hasta 5 ejemplos cada una)
        - Guarda matriz de confusión como 'matriz_de_confusion.png'
        - Imprime estadísticas sobre distribución de datos y progreso de procesamiento
    Notas:
        - Las imágenes se desnormalizan automáticamente usando estadísticas de ImageNet
        - Soporta formatos de imagen CHW y HWC
        - Maneja casos con o sin máscaras de verdad fundamental
        - Limitado a 5 ejemplos por categoría para prevenir generación excesiva de archivos
        - Usa matplotlib para visualización y requiere sklearn para matriz de confusión
    Raises:
        Exception: Captura y reporta errores durante procesamiento de imágenes o guardado de archivos
                  pero continúa la ejecución para las visualizaciones restantes
    """
    
    # Crear directorio principal si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    # Crear directorio de examples
    examples_dir = os.path.join(output_dir, 'examples')
    os.makedirs(examples_dir, exist_ok=True)
    
    print(f"Creando visualizaciones en: {examples_dir}")
    
    # Convertir a numpy arrays si no lo son
    images = np.array(images)
    labels = np.array(labels)
    scores = np.array(scores)
    predictions = (scores >= threshold).astype(int)
    
    print(f"Total de imágenes: {len(images)}")
    print(f"Distribución de etiquetas - Normal: {np.sum(labels == 0)}, Anomalía: {np.sum(labels == 1)}")
    print(f"Distribución de predicciones - Normal: {np.sum(predictions == 0)}, Anomalía: {np.sum(predictions == 1)}")
    
    # Índices para cada categoría (TP, TN, FP, FN)
    true_positive = np.where((predictions == 1) & (labels == 1))[0]
    true_negative = np.where((predictions == 0) & (labels == 0))[0]
    false_positive = np.where((predictions == 1) & (labels == 0))[0]
    false_negative = np.where((predictions == 0) & (labels == 1))[0]
    
    print(f"True Positives: {len(true_positive)}")
    print(f"True Negatives: {len(true_negative)}")
    print(f"False Positives: {len(false_positive)}")
    print(f"False Negatives: {len(false_negative)}")
    
    # Limitar número de ejemplos
    max_examples = 5
    categories = [
        ('true_positive', true_positive[:max_examples]),
        ('true_negative', true_negative[:max_examples]),
        ('false_positive', false_positive[:max_examples]),
        ('false_negative', false_negative[:max_examples])
    ]
    
    # Determinar si tenemos máscaras disponibles
    has_masks = len(masks) > 0 and any(mask is not None for mask in masks)
    print(f"Máscaras disponibles: {has_masks}")
    
    # Visualizar ejemplos por categoría
    for category_name, indices in categories:
        if len(indices) == 0:
            print(f"No hay ejemplos para {category_name}")
            continue
            
        print(f"Creando visualización para {category_name} con {len(indices)} ejemplos")
        
        # Determinar el número de subplots (2 o 3 columnas por muestra)
        n_cols = 3 if has_masks else 2
        fig_width = 4 * len(indices) * n_cols
        fig_height = 4
        
        plt.figure(figsize=(fig_width, fig_height))
        
        for i, idx in enumerate(indices):
            try:
                # Imagen original
                subplot_idx = i * n_cols + 1
                plt.subplot(1, len(indices) * n_cols, subplot_idx)
                
                img = images[idx]
                if img.shape[0] == 3:  # Si está en formato CHW
                    img = img.transpose(1, 2, 0)  # Convertir a HWC
                
                # Desnormalizar la imagen
                img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                img = np.clip(img, 0, 1)
                
                plt.imshow(img)
                plt.title(f"Original\nScore: {scores[idx]:.4f}\nLabel: {int(labels[idx])}")
                plt.axis('off')
                
                # Máscara de ground truth (si está disponible)
                if has_masks and idx < len(masks) and masks[idx] is not None:
                    plt.subplot(1, len(indices) * n_cols, subplot_idx + 1)
                    mask = masks[idx]
                    if hasattr(mask, 'squeeze'):
                        mask = mask.squeeze()
                    elif isinstance(mask, np.ndarray) and mask.ndim > 2:
                        mask = np.squeeze(mask)
                    
                    plt.imshow(mask, cmap='gray')
                    plt.title("Ground Truth")
                    plt.axis('off')
                    
                    # Predicción
                    plt.subplot(1, len(indices) * n_cols, subplot_idx + 2)
                else:
                    # Sin máscara, solo predicción
                    plt.subplot(1, len(indices) * n_cols, subplot_idx + 1)
                
                # Mostrar predicción
                if predictions[idx] == 1:
                    plt.text(0.5, 0.5, "ANOMALY", ha='center', va='center', 
                             fontsize=16, color='red', weight='bold',
                             transform=plt.gca().transAxes)
                else:
                    plt.text(0.5, 0.5, "NORMAL", ha='center', va='center', 
                             fontsize=16, color='green', weight='bold',
                             transform=plt.gca().transAxes)
                
                plt.title(f"Prediction\nThreshold: {threshold:.4f}")
                plt.axis('off')
                
            except Exception as e:
                print(f"Error procesando imagen {idx} en categoría {category_name}: {e}")
                continue
        
        plt.suptitle(f"{category_name.replace('_', ' ').title()}", fontsize=16, y=0.98)
        plt.tight_layout()
        
        # Guardar figura
        filename = os.path.join(examples_dir, f'{category_name}.png')
        try:
            plt.savefig(filename, dpi=150, bbox_inches='tight')
            print(f"Guardado: {filename}")
        except Exception as e:
            print(f"Error guardando {filename}: {e}")
        finally:
            plt.close()  # Cerrar figura para liberar memoria
    
    # Matriz de confusión
    try:
        from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
        cm = confusion_matrix(labels, predictions)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Normal', 'Anomalía'])
        
        plt.figure(figsize=(8, 6))
        disp.plot(cmap='Blues')
        plt.title('Matriz de Confusión')
        
        cm_filename = os.path.join(output_dir, 'matriz_de_confusion.png')
        plt.savefig(cm_filename, dpi=150, bbox_inches='tight')
        print(f"Matriz de confusión guardada: {cm_filename}")
        plt.close()
        
    except Exception as e:
        print(f"Error creando matriz de confusión: {e}")
    
    print(f"Visualizaciones completadas en: {examples_dir}")


In [10]:
def visualize_anomaly_maps(model, test_loader, device, output_dir, threshold):
    """
    Visualiza los resultados de detección de anomalías generando gráficos comparativos de imágenes originales,
    reconstrucciones y mapas de anomalía.
    
    Esta función procesa datos de prueba a través de un modelo autoencoder para generar mapas de anomalía
    basados en errores de reconstrucción. Crea gráficos de visualización que muestran imágenes originales,
    imágenes reconstruidas, mapas de error por píxel y máscaras de verdad fundamental (si están disponibles).
    
    Args:
        model: Modelo autoencoder entrenado para detección de anomalías
        test_loader: DataLoader que contiene imágenes de prueba con etiquetas y máscaras opcionales
        device: Dispositivo PyTorch (CPU/GPU) para inferencia del modelo
        output_dir (str): Ruta del directorio donde se guardarán las visualizaciones de mapas de anomalía
        threshold: Valor umbral para clasificación de anomalías (actualmente no utilizado)
        model_type: Tipo de modelo que se está utilizando (actualmente no utilizado)
        
    Returns:
        None: La función guarda los gráficos de visualización en disco
        
    Notas:
        - Crea hasta 5 muestras cada una para las categorías normal y anomalía
        - Soporta datasets con o sin máscaras de verdad fundamental
        - Las imágenes se desnormalizan usando estadísticas de ImageNet antes de la visualización
        - Los mapas de error se calculan como diferencias absolutas por píxel entre imágenes originales y reconstruidas
        - Los gráficos de salida se guardan como archivos PNG en el subdirectorio 'anomaly_maps'
        - La función espera formato de lote: (imágenes, etiquetas) o (imágenes, etiquetas, máscaras)
    """

    os.makedirs(os.path.join(output_dir, 'anomaly_maps'), exist_ok=True)
    model.eval()
    
    # Seleccionar algunas imágenes para visualización
    samples_seen = {'normal': 0, 'anomaly': 0}
    max_samples = 5
    
    with torch.no_grad():
        for batch in test_loader:
            # Manejar batch con o sin máscara
            if len(batch) == 3:
                images, labels, masks = batch
                has_masks = True
            else:
                images, labels = batch
                masks = None
                has_masks = False
            
            images = images.to(device)
            batch_size = images.size(0)
            
            # Modelo autoencoder
            
            reconstructed, _ = model(images)
            # Calcular error de reconstrucción por píxel
            error_maps = torch.abs(images - reconstructed)
            # Normalizar mapas de error para visualización
            error_maps_mean = error_maps.mean(dim=1)  # Promediar a través de los canales
                
            # Convertir a numpy para visualización
            images_np = images.cpu().numpy()
            error_maps_np = error_maps_mean.cpu().numpy()
            reconstructed_np = reconstructed.cpu().numpy()
                
            for i in range(batch_size):
                label = labels[i].item()
                category = 'normal' if label == 0 else 'anomaly'
                    
                if samples_seen[category] < max_samples:
                    # Determinar número de subplots
                    n_cols = 4 if has_masks else 3
                    fig, axes = plt.subplots(1, n_cols, figsize=(5 * n_cols, 5))
                        
                    # Imagen original
                    img = images_np[i].transpose(1, 2, 0)
                    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                    img = np.clip(img, 0, 1)                        
                    axes[0].imshow(img)
                    axes[0].set_title("Original")
                    axes[0].axis('off')
                        
                    # Imagen reconstruida
                    rec_img = reconstructed_np[i].transpose(1, 2, 0)
                    rec_img = rec_img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                    rec_img = np.clip(rec_img, 0, 1)
                    axes[1].imshow(rec_img)
                    axes[1].set_title("Reconstrucción")
                    axes[1].axis('off')
                        
                    # Mapa de error
                    error_map = error_maps_np[i]
                    vmax = np.max(error_map) if np.max(error_map) > 0 else 1
                    im = axes[2].imshow(error_map, cmap='jet', vmin=0, vmax=vmax)
                    axes[2].set_title(f"Mapa de Anomalía\n(Score: {error_maps_mean[i].mean().item():.4f})")
                    axes[2].axis('off')
                        
                    # Máscara de ground truth (si está disponible)
                    if has_masks and masks is not None:
                        mask_np = masks[i].squeeze().cpu().numpy()
                        axes[3].imshow(mask_np, cmap='gray')
                        axes[3].set_title("Ground Truth Mask")
                        axes[3].axis('off')
                        
                    plt.colorbar(im, ax=axes[2])
                    plt.tight_layout()
                    plt.savefig(os.path.join(output_dir, 'anomaly_maps', f'{category}_{samples_seen[category]}.png'))
                    plt.close()
                        
                    samples_seen[category] += 1
            
            if all(count >= max_samples for count in samples_seen.values()):
                break


### Funciones de Evaluación

In [11]:
def eval(model, final_test_loader, device, output_dir, model_name, model_type='autoencoder'):
    """
    Evalúa un modelo de detección de anomalías en el conjunto de prueba final usando puntuación consistente.
    
    Esta función realiza una evaluación integral de un modelo de detección de anomalías,
    calculando varias métricas, generando gráficos de visualización y guardando resultados.
    
    Args:
        model: El modelo de detección de anomalías entrenado a evaluar. Debe tener el método
               'detectar_anomalia_consistente' o una pasada hacia adelante estándar.
        final_test_loader: DataLoader que contiene el dataset de prueba con imágenes,
                          etiquetas y máscaras.
        device: Objeto torch.device que especifica GPU o CPU para el cómputo.
        output_dir (str): Ruta del directorio base donde se guardarán los resultados de evaluación.
        model_name (str): Nombre del modelo, usado para crear subdirectorios.
        model_type (str, opcional): Tipo de modelo que se está evaluando. Por defecto es 'autoencoder'.
    
    Returns:
        dict: Diccionario que contiene las métricas de evaluación con las siguientes claves:
            - 'roc_auc': Área bajo la curva ROC
            - 'avg_precision': Puntuación de precisión promedio
            - 'accuracy': Precisión de clasificación usando el umbral óptimo
            - 'precision': Puntuación de precisión
            - 'recall': Puntuación de recall
            - 'f1': Puntuación F1
            - 'threshold': Valor del umbral óptimo
            - 'scores': Lista de todas las puntuaciones de anomalía
            - 'labels': Lista de todas las etiquetas verdaderas
    
    Efectos Secundarios:
        - Crea el directorio de salida si no existe
        - Guarda gráficos de evaluación (curva ROC, curva PR, distribución de puntuaciones) como PNG
        - Imprime resultados detallados de evaluación en consola
        - Llama a visualize_examples() para generar visualizaciones de ejemplo
        - Establece el modelo en modo de evaluación
    
    Nota:
        La función usa metodología de puntuación consistente, prefiriendo el método
        'detectar_anomalia_consistente' del modelo si está disponible, de lo contrario
        recurre al error de reconstrucción MSE.
    """
    
    output_dir = output_dir+model_name
    ensure_directory_exists(output_dir)
    
    model.eval()
    all_scores = []
    all_labels = []
    all_images = []
    all_masks = []
    
    print("\n" + "="*60)
    print(" REALIZANDO EVALUACIÓN FINAL CON SCORING CONSISTENTE")
    print("="*60)
    
    with torch.no_grad():
        for images, labels, masks in tqdm(final_test_loader, desc="Evaluación final"):
            images = images.to(device)
            
            # Usar scoring consitente
            batch_scores = []
            for i in range(images.size(0)):
                single_img = images[i:i+1]  # Mantener dimensión batch
                    
                # Usar el método de detección consistente
                if hasattr(model, 'detectar_anomalia_consistente'):
                    score = model.detectar_anomalia_consistente(single_img)
                    batch_scores.append(score.item())
                else:
                    # Fallback al método original si no está disponible
                    reconstructed, _ = model(single_img)
                    error = torch.mean((single_img - reconstructed) ** 2)
                    batch_scores.append(error.item())
                
            scores = batch_scores
            
            all_scores.extend(scores)
            all_labels.extend(labels.numpy())
            all_images.extend(images.cpu().numpy())
            all_masks.extend(masks.cpu().numpy() if hasattr(masks, 'cpu') else masks)
    
    # =========================
    # Calcular Métricas Finales
    # =========================

    # Calcular métricas finales
    fpr, tpr, thresholds = roc_curve(all_labels, all_scores)
    avg_precision = average_precision_score(all_labels, all_scores)
    roc_auc = auc(fpr, tpr)
    
    # Encontrar mejor threshold
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    
    # Calcular accuracy con threshold óptimo
    predictions = (np.array(all_scores) > optimal_threshold).astype(int)
    accuracy = np.mean(predictions == all_labels)
    
    # Calcular precision, recall, F1
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, predictions, average='binary'
    )
    
    precision_curve, recall_curve, _ = precision_recall_curve(all_labels, all_scores)

    print("\n" + "="*60)
    print(" RESULTADOS FINALES (SCORING CONSISTENTE)")
    print("="*60)
    print(f"ROC AUC:        {roc_auc:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Accuracy:       {accuracy:.4f}")
    print(f"Precision:      {precision:.4f}")
    print(f"Recall:         {recall:.4f}")
    print(f"F1-Score:       {f1:.4f}")
    print(f"Optimal Threshold: {optimal_threshold:.6f}")
    print("="*60)
    
    # =================
    # Visualizar curvas
    # =================
    plt.figure(figsize=(15, 5))

    # ROC Curve
    plt.subplot(1, 3, 1)
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Consistent Scoring)')
    plt.legend(loc="lower right")

    # Precision-Recall Curve
    plt.subplot(1, 3, 2)
    plt.plot(recall_curve, precision_curve, color='blue', lw=2, label=f'PR curve (AP = {avg_precision:.3f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="upper right")
    
    # Score Distribution
    plt.subplot(1, 3, 3)
    normal_scores = [score for score, label in zip(all_scores, all_labels) if label == 0]
    anomaly_scores = [score for score, label in zip(all_scores, all_labels) if label == 1]
    
    plt.hist(normal_scores, bins=50, alpha=0.7, label='Normal', color='green', density=True)
    plt.hist(anomaly_scores, bins=50, alpha=0.7, label='Anomaly', color='red', density=True)
    plt.axvline(optimal_threshold, color='black', linestyle='--', label=f'Threshold: {optimal_threshold:.3f}')
    plt.xlabel('Anomaly Score')
    plt.ylabel('Density')
    plt.title('Score Distribution')
    plt.legend()

    plt.tight_layout()
    curves_filename = os.path.join(output_dir, 'evaluacion_final_consistente.png')
    plt.savefig(curves_filename, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f' Gráficos de evaluación guardados en: {curves_filename}')
    
    # ===================
    # Visualizar ejemplos
    # ===================
    visualize_examples(all_images, all_labels, all_scores, all_masks, optimal_threshold, output_dir, model_type)

    return {
        'roc_auc': float(roc_auc),
        'avg_precision': float(avg_precision),
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'threshold': float(optimal_threshold),
        'scores': all_scores,
        'labels': all_labels
    }

### Funciones auxiliares

In [12]:
# Función auxiliar para verificar y crear directorios
def ensure_directory_exists(path):
    """
    Asegura que un directorio exista, creándolo si es necesario.
    Esta función intenta crear un directorio en la ruta especificada. Si el directorio
    ya existe, no se genera ningún error debido al parámetro exist_ok=True. La función
    proporciona retroalimentación en consola sobre el éxito o fallo de la operación.
    Args:
        path (str): La ruta del sistema de archivos donde el directorio debe existir o ser creado.
    Returns:
        bool: True si el directorio existe o fue creado exitosamente, False si ocurrió un error.
    Raises:
        Imprime mensajes de error en consola pero no levanta excepciones.
    Example:
        >>> ensure_directory_exists("/ruta/al/nuevo/directorio")
        Directorio verificado/creado: /ruta/al/nuevo/directorio
        True
        >>> ensure_directory_exists("/ruta/invalida")
        Error creando directorio /ruta/invalida: [Errno 13] Permiso denegado
        False
    """
    
    try:
        os.makedirs(path, exist_ok=True)
        print(f"Directorio verificado/creado: {path}")
        return True
    except Exception as e:
        print(f"Error creando directorio {path}: {e}")
        return False

In [13]:
# Función para analizar el balance de un datset
def analyze_dataset_balance(dataset, name="Dataset"):
    """
    Analiza y muestra el balance de clases de un dataset de clasificación binaria.
    Esta función extrae las etiquetas de un dataset, cuenta las ocurrencias de cada clase,
    e imprime estadísticas detalladas sobre la distribución de muestras normales vs anomalías.
    Args:
        dataset: Un dataset iterable donde cada elemento retorna una tupla de (datos, etiqueta, metadatos).
                Se espera que la etiqueta esté en el índice 1 y sea convertible a int.
        name (str, opcional): Nombre del dataset para propósitos de visualización. Por defecto "Dataset".
    Returns:
        Counter: Un objeto Counter que contiene el conteo de cada etiqueta de clase.
                La clase 0 representa muestras normales, la clase 1 representa anomalías.
    Imprime:
        - Número total de muestras
        - Conteo y porcentaje de muestras normales (clase 0)
        - Conteo y porcentaje de muestras con anomalías (clase 1)  
        - Proporción de muestras normales respecto a anomalías
    Ejemplo:
        >>> counts = analyze_dataset_balance(my_dataset, "Conjunto de Entrenamiento")
         ANÁLISIS DE BALANCE - Conjunto de Entrenamiento
           Total muestras: 1000
           Normal: 800 (80.0%)
           Anomalía: 200 (20.0%)
           Ratio Normal:Anomalía = 1:0.25
    """

    labels = []
    for i in range(len(dataset)):
        _, label, _ = dataset[i]
        labels.append(int(label))
  
    class_counts = Counter(labels)
    total = len(labels)
  
    print(f"\n ANÁLISIS DE BALANCE - {name}")
    print(f"   Total muestras: {total}")
    print(f"   Normal: {class_counts[0]} ({class_counts[0]/total*100:.1f}%)")
    print(f"   Anomalía: {class_counts[1]} ({class_counts[1]/total*100:.1f}%)")
    print(f"   Ratio Normal:Anomalía = 1:{class_counts[1]/class_counts[0]:.2f}")
    
    return class_counts

In [14]:
def convert_numpy_types(obj):
    """
    Convierte recursivamente tipos de datos NumPy a tipos nativos de Python.
    Esta función recorre estructuras de datos anidadas (diccionarios y listas)
    y convierte tipos específicos de NumPy a sus tipos nativos de Python correspondientes
    para mejorar la compatibilidad con la serialización JSON y otras operaciones.
    Args:
        obj: El objeto a convertir. Puede ser un diccionario, lista, array de NumPy,
             escalar de NumPy, o cualquier otro tipo.
    Returns:
        El objeto convertido con tipos NumPy reemplazados por tipos nativos de Python:
        - np.integer -> int
        - np.floating -> float  
        - np.ndarray -> list
        - Los diccionarios y listas se procesan recursivamente
        - Otros tipos se devuelven sin cambios
    Examples:
        >>> import numpy as np
        >>> data = {'array': np.array([1, 2, 3]), 'value': np.int64(42)}
        >>> convert_numpy_types(data)
        {'array': [1, 2, 3], 'value': 42}
    """
    
    if isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj


### Cuerpo principal

In [15]:
# Variables de configuración
data_path = 'data/'  # Ruta al dataset MVTec AD
output_dir = 'models/'  # Ruta para guardar el modelo entrenado
reports_path = 'reports/'  # Ruta para guardar los reportes e imágenes
logs_dir = 'logs/' # Directorio de logs para tensorboard

# Lista de todas las categorías en MVTec AD
categories = [
    "bottle",
    "cable",
    "capsule",
    "carpet",
    "grid",
    "hazelnut",
    "leather",
    "metal_nut",
    "pill",
    "screw",
    "tile",
    "toothbrush",
    "transistor",
    "wood",
    "zipper",
]

In [16]:
# Crear directorios de salida en caso de que no existan
os.makedirs(output_dir, exist_ok=True)
os.makedirs(reports_path, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)

In [17]:
# Semilla para reproducibilidad de los experimentos
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [18]:
# Si tenemos disponible GPU, lo usamos
# Chequeamos si tenemos disponible GPU (CUDA)
if torch.cuda.is_available():
    device = "cuda"
# Chequeamos si tenemos disponible aceleración por hardware en un chip de Apple (MPS)
elif torch.backends.mps.is_available():
    device = "mps"
# Por defecto usamos CPU
else:
    device = "cpu"

print(f" Usando dispositivo: {device}")

 Usando dispositivo: cuda


In [19]:
# Definir transformaciones
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Transformación para máscaras (sin normalización)
mask_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [20]:
# Crear Datasets
print("\n" + "="*60)
print("CARGANDO DATASETS MVTec AD")
print("="*60)

# Crear datasets para todas las categorías
train_datasets = []
test_datasets = []

for category in categories:
    try:
        train_dataset = MVTecDataset(
            root_path=data_path,
            category=category,
            is_train=True,
            transform=train_transform,
            mask_transform=mask_transform
        )

        test_dataset = MVTecDataset(
            root_path=data_path,
            category=category,
            is_train=False,
            transform=test_transform,
            mask_transform=mask_transform
        )
                
        train_datasets.append(train_dataset)
        test_datasets.append(test_dataset)
        print(f"{category:12} | Train: {len(train_dataset):3} | Test: {len(test_dataset):3}")
    except Exception as e:
        print(f" Error al cargar la categoría {category}: {e}")


CARGANDO DATASETS MVTec AD
bottle       | Train: 209 | Test:  83
cable        | Train: 224 | Test: 150
capsule      | Train: 219 | Test: 132
carpet       | Train: 280 | Test: 117
grid         | Train: 264 | Test:  78
hazelnut     | Train: 391 | Test: 110
leather      | Train: 245 | Test: 124
metal_nut    | Train: 220 | Test: 115
pill         | Train: 267 | Test: 167
screw        | Train: 320 | Test: 160
tile         | Train: 230 | Test: 117
toothbrush   | Train:  60 | Test:  42
transistor   | Train: 213 | Test: 100
wood         | Train: 247 | Test:  79
zipper       | Train: 240 | Test: 151


In [21]:
# Dividir datasets de test en validación y test final
val_datasets, final_test_datasets = split_test_datasets(test_datasets, categories, test_size=0.5)

# Combinar datasets
train_dataset = ConcatDataset(train_datasets)  # Unir todos los datasets de entrenamiento
val_dataset = ConcatDataset(val_datasets)      # Unir todos los datasets de validación
final_test_dataset = ConcatDataset(final_test_datasets)  # Unir todos los datasets de test final

print(f"\n RESUMEN DE DATASETS:")
print(f"   Entrenamiento: {len(train_dataset):4} imágenes")
print(f"   Validación:    {len(val_dataset):4} imágenes")
print(f"   Test final:    {len(final_test_dataset):4} imágenes")
print(f"   Total:         {len(train_dataset) + len(val_dataset) + len(final_test_dataset):4} imágenes")



DIVISIÓN DE DATASETS DE TEST
bottle       | Total:  83 | Normal:  20 | Anomalía:  63 | Val:  41 | Test:  42
cable        | Total: 150 | Normal:  58 | Anomalía:  92 | Val:  75 | Test:  75
capsule      | Total: 132 | Normal:  23 | Anomalía: 109 | Val:  66 | Test:  66
carpet       | Total: 117 | Normal:  28 | Anomalía:  89 | Val:  58 | Test:  59
grid         | Total:  78 | Normal:  21 | Anomalía:  57 | Val:  39 | Test:  39
hazelnut     | Total: 110 | Normal:  40 | Anomalía:  70 | Val:  55 | Test:  55
leather      | Total: 124 | Normal:  32 | Anomalía:  92 | Val:  62 | Test:  62
metal_nut    | Total: 115 | Normal:  22 | Anomalía:  93 | Val:  57 | Test:  58
pill         | Total: 167 | Normal:  26 | Anomalía: 141 | Val:  83 | Test:  84
screw        | Total: 160 | Normal:  41 | Anomalía: 119 | Val:  80 | Test:  80
tile         | Total: 117 | Normal:  33 | Anomalía:  84 | Val:  58 | Test:  59
toothbrush   | Total:  42 | Normal:  12 | Anomalía:  30 | Val:  21 | Test:  21
transistor   | Total: 

#### Analizamos el balanceo de los datasets

In [22]:
# Vamos a analizar como se encuentran balanceados los datasets
analyze_dataset_balance(val_dataset, "Validación ANTES")
analyze_dataset_balance(final_test_dataset, "Test ANTES")


 ANÁLISIS DE BALANCE - Validación ANTES
   Total muestras: 859
   Normal: 231 (26.9%)
   Anomalía: 628 (73.1%)
   Ratio Normal:Anomalía = 1:2.72

 ANÁLISIS DE BALANCE - Test ANTES
   Total muestras: 866
   Normal: 236 (27.3%)
   Anomalía: 630 (72.7%)
   Ratio Normal:Anomalía = 1:2.67


Counter({1: 630, 0: 236})

#### Balanceamos los datasets y creamos los dataloaders

In [23]:
# Defino el tamaño de los lotes
batch_size = 16 # Tamaño de los lotes, mas grande requiere mas VRAM o RAM

print(f"\n CREANDO DATALOADERS BALANCEADOS...")
print("="*60)

# ENTRENAMIMENTO: Para autoencoders, generalmente NO se balancea
# porque se entrenan principalmente con datos normales
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=8
)
print(f" TRAIN LOADER (Sin balancear - normal para autoencoders)")
print(f"   Tamaño: {len(train_dataset)} muestras")

# VALIDACIÓN: Balancear para evaluación justa
print(f"\n VALIDATION LOADER (Balanceado)")
val_loader = crear_dataloader_balanceado(
    val_dataset, 
    batch_size=batch_size, 
    method='weighted',  # Opciones: 'weighted', 'oversample', 'undersample'
    num_workers=8
)

#TEST FINAL: Balancear para evaluación final justa  
print(f"\n FINAL TEST LOADER (Balanceado)")
final_test_loader = crear_dataloader_balanceado(
    final_test_dataset, 
    batch_size=batch_size, 
    method='weighted',  # Usar el mismo método que validación
    num_workers=8
)

print(f"\n DataLoaders balanceados creados con batch_size={batch_size}")
print("="*60)



 CREANDO DATALOADERS BALANCEADOS...
 TRAIN LOADER (Sin balancear - normal para autoencoders)
   Tamaño: 3629 muestras

 VALIDATION LOADER (Balanceado)
   Distribución original: Normal=231, Anomalía=628
   Método: Weighted Sampling
   Pesos: Normal=1.859, Anomalía=0.684

 FINAL TEST LOADER (Balanceado)
   Distribución original: Normal=236, Anomalía=630
   Método: Weighted Sampling
   Pesos: Normal=1.835, Anomalía=0.687

 DataLoaders balanceados creados con batch_size=16


In [24]:
# Configuración del modelo

model_name = "SwinTransformerAutoencoder_2" # Nombre para el modelo asi se guarda en el output_dir y # en los logs

pretrained = True # Usar pre-entrenado
lr = 2e-5 # Learning rate para el optimizador
wd = 5e-4 # Weight decay para regularización
num_epochs = 200 # Número de épocas para entrenamiento

print(f"\n CONFIGURACIÓN DEL MODELO:")
print(f"   Tipo:          Autoencoder con Swin Transformer")
print(f"   Pre-entrenado: {pretrained}")
print(f"   Learning rate: {lr}")
print(f"   Weight decay:  {wd}")
print(f"   Batch size:    {batch_size}")
print(f"   Device:        {device}")
print(f"   Modelo:        {model_name}")
print(f"   Épocas:        {num_epochs}")
print(f"   Categorías:    TODAS ({len(categories)} categorías combinadas)")



 CONFIGURACIÓN DEL MODELO:
   Tipo:          Autoencoder con Swin Transformer
   Pre-entrenado: True
   Learning rate: 2e-05
   Weight decay:  0.0005
   Batch size:    16
   Device:        cuda
   Modelo:        SwinTransformerAutoencoder_2
   Épocas:        200
   Categorías:    TODAS (15 categorías combinadas)


In [25]:
# Antes de entrenar lanzamos tensorboard
%load_ext tensorboard
%tensorboard --logdir=./logs --host 0.0.0.0 --port 6006

### Para acceder a tensorboard:

http://localhost:6006


In [27]:
# Creamos el modelo
model = SwinTransformerAutoencoder(pretrained=pretrained).to(device)

criterion = CombinedReconstructionLoss(
    mse_weight=0.3, # 0.4,
    ssim_weight=0.8,    # 0.5,
    l1_weight=0.2  #0.1
    )

In [28]:
# Optimizador y scheduler
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd) # AdamW es una variante de Adam con decaimiento de peso
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=5) # Reduce el learning rate si la métrica no mejora
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)


In [29]:
# Entrenamos el modelo
print(f"\n INICIANDO ENTRENAMIENTO DEL MODELO ...")
print("="*60)

model = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,  #  Usando val_loader en lugar de test_loader
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=num_epochs,
    device=device,
    output_dir=output_dir,
    logs_dir=logs_dir,
    model_name=model_name,
)

print("\n Entrenamiento finalizado!")


 INICIANDO ENTRENAMIENTO DEL MODELO ...

 INICIANDO ENTRENAMIENTO
   Épocas: 200
   Función de pérdida: MSE + SSIM + L1
   Dispositivo: cuda


Epoch 1/200 (val): 100%|██████████| 54/54 [00:03<00:00, 15.85it/s]


      Nuevo mejor modelo guardado! AUC: 0.5016

Epoch 1/200:
   Train Loss:
     Total: 1.5265 | MSE: 1.7632 | SSIM: 0.9587 | L1: 1.1527
   Val Loss:
     Total: 1.5125 | MSE: 1.7382 | SSIM: 0.9529 | L1: 1.1437
   ROC AUC (Combined Scoring): 0.5016 | Best: 0.5016
   Learning Rate: 2.00e-05
------------------------------------------------------------


Epoch 2/200 (val): 100%|██████████| 54/54 [00:03<00:00, 16.16it/s]


      AUC no mejorado. Paciencia: 1/201

Epoch 2/200:
   Train Loss:
     Total: 1.4970 | MSE: 1.7202 | SSIM: 0.9422 | L1: 1.1360
   Val Loss:
     Total: 1.4675 | MSE: 1.6463 | SSIM: 0.9414 | L1: 1.1027
   ROC AUC (Combined Scoring): 0.4760 | Best: 0.5016
   Learning Rate: 2.00e-05
------------------------------------------------------------


...

      AUC no mejorado. Paciencia: 83/201

Epoch 198/200:
   Train Loss:
     Total: 1.3472 | MSE: 1.4601 | SSIM: 0.8781 | L1: 1.0336
   Val Loss:
     Total: 1.2959 | MSE: 1.3561 | SSIM: 0.8639 | L1: 0.9896
   ROC AUC (Combined Scoring): 0.5057 | Best: 0.5520
   Learning Rate: 2.44e-09
------------------------------------------------------------


Epoch 199/200 (val): 100%|██████████| 54/54 [00:03<00:00, 16.15it/s]


      AUC no mejorado. Paciencia: 84/201

Epoch 199/200:
   Train Loss:
     Total: 1.3484 | MSE: 1.4616 | SSIM: 0.8786 | L1: 1.0351
   Val Loss:
     Total: 1.3380 | MSE: 1.4573 | SSIM: 0.8685 | L1: 1.0299
   ROC AUC (Combined Scoring): 0.4952 | Best: 0.5520
   Learning Rate: 2.44e-09
------------------------------------------------------------


Epoch 200/200 (val): 100%|██████████| 54/54 [00:03<00:00, 16.53it/s]


      AUC no mejorado. Paciencia: 85/201

Epoch 200/200:
   Train Loss:
     Total: 1.3462 | MSE: 1.4581 | SSIM: 0.8781 | L1: 1.0317
   Val Loss:
     Total: 1.3143 | MSE: 1.4306 | SSIM: 0.8531 | L1: 1.0133
   ROC AUC (Combined Scoring): 0.5419 | Best: 0.5520
   Learning Rate: 2.44e-09
------------------------------------------------------------

 ENTRENAMIENTO COMPLETADO!
   Mejor AUC alcanzado: 0.5520

 Entrenamiento finalizado!


In [30]:
# Evaluación Final con el conjunto de test independiente
print(f"\n CARGANDO MEJOR MODELO PARA EVALUACIÓN FINAL...")

# Cargo el mejor modelo autoencoder
model.load_state_dict(torch.load(os.path.join(output_dir, f'{model_name}.pth')))

# Evaluación final
resultados_finales = eval(model, final_test_loader, device, reports_path, model_name, model_type='autoencoder')




 CARGANDO MEJOR MODELO PARA EVALUACIÓN FINAL...
Directorio verificado/creado: reports/SwinTransformerAutoencoder_2

 REALIZANDO EVALUACIÓN FINAL CON SCORING CONSISTENTE


Evaluación final: 100%|██████████| 55/55 [00:08<00:00,  6.43it/s]



 RESULTADOS FINALES (SCORING CONSISTENTE)
ROC AUC:        0.4977
Average Precision: 0.5485
Accuracy:       0.5543
Precision:      0.5345
Recall:         0.9977
F1-Score:       0.6961
Optimal Threshold: 0.542667
 Gráficos de evaluación guardados en: reports/SwinTransformerAutoencoder_2/evaluacion_final_consistente.png
Creando visualizaciones en: reports/SwinTransformerAutoencoder_2/examples
Total de imágenes: 866
Distribución de etiquetas - Normal: 423, Anomalía: 443
Distribución de predicciones - Normal: 38, Anomalía: 828
True Positives: 443
True Negatives: 38
False Positives: 385
False Negatives: 0
Máscaras disponibles: True
Creando visualización para true_positive con 5 ejemplos
Guardado: reports/SwinTransformerAutoencoder_2/examples/true_positive.png
Creando visualización para true_negative con 5 ejemplos
Guardado: reports/SwinTransformerAutoencoder_2/examples/true_negative.png
Creando visualización para false_positive con 5 ejemplos
Guardado: reports/SwinTransformerAutoencoder_2/e

<Figure size 800x600 with 0 Axes>

In [ ]:
#print(resultados_finales)
threshold = resultados_finales['threshold']

In [ ]:
# Guardar resultados finales
results_to_save = {k: v for k, v in resultados_finales.items() if k not in ['scores', 'labels']}
results_to_save = convert_numpy_types(results_to_save)  # Convertir tipos numpy
results_file = os.path.join(reports_path+model_name, 'resultados_finales.json')

with open(results_file, 'w') as f:
    json.dump(results_to_save, f, indent=2)

print(f"\n Resultados guardados en: {results_file}")
print("\n ¡PROCESO COMPLETADO EXITOSAMENTE!")

In [ ]:
# Visualizar mapas de anomalías
print("Generando mapas de anomalías...")
visualize_anomaly_maps(
    model=model,
    test_loader=final_test_loader,
    device=device,
    output_dir=reports_path+model_name,
    threshold=threshold
)